### same as the article on the website, but using averagepooling instead of max pooling

- this is roughly based upon https://github.com/fchollet/keras/blob/master/examples/imdb_fasttext.py

In [1]:
import os
import sys
import numpy as np
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.layers import Dense, Input, Flatten
from keras.layers import Conv1D, Dropout, Embedding, GlobalMaxPooling1D, GlobalAvgPool1D
from keras.models import Model,Sequential

from sklearn.feature_extraction.text import CountVectorizer
from sklearn import preprocessing

from helpers import (files_helper, 
                     metrics_helper, 
                     tags_helper,
                     texts_helper)

Using TensorFlow backend.


In [2]:
SEED=np.random.randint(1,1000)
SEED

495

In [3]:
np.random.seed(SEED)

In [4]:
MAX_NB_WORDS = 20000
MAX_SEQUENCE_LENGTH = 1000
VALIDATION_SPLIT = 0.2
EMBEDDING_DIM=100
LABELS_MIN_DOC_COUNT = int(10)
BATCH_SIZE=32
NUM_EPOCHS=10
TOKENIZER_FILTERS='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n' # default keras filters

In [5]:
embeddings_index = files_helper.read_glove(d=EMBEDDING_DIM)

In [6]:
texts,labels_index,labels_0 = files_helper.read_20_newgroup_files()       

In [7]:
tokenizer = Tokenizer(num_words=MAX_NB_WORDS,
                      filters=TOKENIZER_FILTERS)
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)
word_index = tokenizer.word_index
inverse_word_index = texts_helper.build_inverse_word_index(word_index)

In [8]:
len(word_index),len(labels_index)

(174074, 20)

In [9]:
[inverse_word_index[idx] for idx in sequences[0]]

['archive',
 'name',
 'atheism',
 'resources',
 'alt',
 'atheism',
 'archive',
 'name',
 'resources',
 'last',
 'modified',
 '11',
 'december',
 '1992',
 'version',
 '1',
 '0',
 'atheist',
 'resources',
 'addresses',
 'of',
 'atheist',
 'organizations',
 'usa',
 'freedom',
 'from',
 'religion',
 'foundation',
 'darwin',
 'fish',
 'bumper',
 'stickers',
 'and',
 'assorted',
 'other',
 'atheist',
 'are',
 'available',
 'from',
 'the',
 'freedom',
 'from',
 'religion',
 'foundation',
 'in',
 'the',
 'us',
 'write',
 'to',
 'p',
 'o',
 'box',
 '750',
 'madison',
 'wi',
 'telephone',
 '608',
 '256',
 '8900',
 'evolution',
 'designs',
 'evolution',
 'designs',
 'sell',
 'the',
 'darwin',
 'fish',
 "it's",
 'a',
 'fish',
 'symbol',
 'like',
 'the',
 'ones',
 'christians',
 'stick',
 'on',
 'their',
 'cars',
 'but',
 'with',
 'feet',
 'and',
 'the',
 'word',
 'darwin',
 'written',
 'inside',
 'the',
 'deluxe',
 '3d',
 'plastic',
 'fish',
 'is',
 '4',
 '95',
 'in',
 'the',
 'us',
 'write',
 'to

In [10]:
data = pad_sequences(sequences,maxlen=MAX_SEQUENCE_LENGTH)

labels_1 = to_categorical(np.asarray(labels_0))

indices = np.arange(data.shape[0])
np.random.shuffle(indices)

data = data[indices]
labels_2 = labels_1[indices]
num_validation_samples = int(VALIDATION_SPLIT * data.shape[0])

X_train = data[:-num_validation_samples]
y_train = labels_2[:-num_validation_samples]
X_val = data[-num_validation_samples:]
y_val = labels_2[-num_validation_samples:]

data.shape,labels_2.shape

((19997, 1000), (19997, 20))

In [11]:
embedding_matrix = np.zeros((len(word_index)+1,EMBEDDING_DIM))

for word,i in word_index.items():
    
    if i >= MAX_NB_WORDS:
        continue
    
    embedding_vector = embeddings_index.get(word)
    
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

In [12]:
embedding_matrix.shape

(174075, 100)

In [13]:
model = Sequential()

model.add(Embedding(len(word_index)+1,
                           EMBEDDING_DIM,
                           weights=[embedding_matrix],
                           input_length=MAX_SEQUENCE_LENGTH,
                           trainable = False))

model.add(GlobalAvgPool1D())

model.add(Dense(64,activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64,activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(len(labels_index), activation='softmax'))

model.compile(loss ='categorical_crossentropy',
             optimizer='adam',
             metrics=['acc'])

In [14]:
%%time

model.fit(X_train,y_train, validation_data=(X_val, y_val),
         epochs=NUM_EPOCHS, batch_size=BATCH_SIZE)

Train on 15998 samples, validate on 3999 samples
Epoch 1/10
15998/15998 [==============================] - 2s - loss: 2.9076 - acc: 0.0881 - val_loss: 2.6476 - val_acc: 0.2086
Epoch 2/10
15998/15998 [==============================] - 2s - loss: 2.5440 - acc: 0.1667 - val_loss: 2.2800 - val_acc: 0.2543
Epoch 3/10
15998/15998 [==============================] - 1s - loss: 2.3282 - acc: 0.2070 - val_loss: 2.0986 - val_acc: 0.3323
Epoch 4/10
15998/15998 [==============================] - 1s - loss: 2.2186 - acc: 0.2413 - val_loss: 2.0061 - val_acc: 0.3551
Epoch 5/10
15998/15998 [==============================] - 2s - loss: 2.1352 - acc: 0.2658 - val_loss: 1.9093 - val_acc: 0.3713
Epoch 6/10
15998/15998 [==============================] - 2s - loss: 2.0636 - acc: 0.2926 - val_loss: 1.8467 - val_acc: 0.3923
Epoch 7/10
15998/15998 [==============================] - 1s - loss: 2.0117 - acc: 0.3078 - val_loss: 1.7929 - val_acc: 0.4096
Epoch 8/10
15998/15998 [==============================] - 2s -